In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import torch
import torch.utils.data
import torchvision
# from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
from icecream import ic
from collections import Counter
import sys

### Load data

In [19]:
path = r"Data/final"
image_addr_list = os.listdir(path)
ic(len(image_addr_list))
data = []
img_num = []
for i in tqdm(image_addr_list):
    full_path = os.path.join(path, i)
    imgobj = Image.open(full_path).convert('L')
    img = np.asarray(imgobj)
    data.append(img)
    img_num.append(i) # Not very useful but good for debugging

ic| len(image_addr_list): 4046


  0%|          | 0/4046 [00:00<?, ?it/s]

In [18]:
data = np.array(data)
ic(data.shape)

ic| data.shape: (4046, 250, 730, 3)


(4046, 250, 730, 3)